In [1]:
import warnings
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
# warnings.filterwarnings('ignore')

from Energy_Models import CNN_LSTM_V2
from Energy_Models import Evaluation_Metrix

from tensorflow.keras import Sequential
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, TimeDistributed


C:\Users\Mohamed\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Reading

In [2]:
#index >> date-time
df = pd.read_csv(r'household_power_consumption Preprocessed.csv',parse_dates=['date_time'], index_col= 'date_time')

# Data Analysis

In [3]:
#  group data by day
data = df.resample('D').sum() 

In [4]:
data.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
date_time,,,,,,,
2006-12-16,1209.176,34.922,93552.53,5180.8,0.0,546.0,4926.0
2006-12-17,3390.460,226.006,345725.32,14398.6,2033.0,4187.0,13341.0
2006-12-18,2203.826,161.792,347373.64,9247.2,1063.0,2621.0,14018.0
2006-12-19,1666.194,150.942,348479.01,7094.0,839.0,7602.0,6197.0
2006-12-20,2225.748,160.998,348923.61,9313.0,0.0,2648.0,14063.0


# Data Preprocessing

In [5]:
# Compute remaining active power
df['sub_metering_remaining'] = (df.Global_active_power * 1000  / 60 ) - (df.Sub_metering_1 + df.Sub_metering_2 + df.Sub_metering_3)

In [6]:
# descriptive statistics
df.describe()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,sub_metering_remaining
count,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06,2.075259e+06
mean,1.089418e+00,1.236871e-01,2.408364e+02,4.618401e+00,1.118474e+00,1.291131e+00,6.448635e+00,9.298722e+00
std,1.054678e+00,1.125933e-01,3.240051e+00,4.433165e+00,6.141460e+00,5.796922e+00,8.433584e+00,9.561278e+00
min,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,-2.400000e+00
25%,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.800000e+00
50%,6.020000e-01,1.000000e-01,2.410000e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00,5.500000e+00
75%,1.526000e+00,1.940000e-01,2.428700e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01,1.036667e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01,1.248333e+02


In [7]:
# resample the data on daily basis
df = df.resample('D').mean()
df.shape

(1442, 8)

In [8]:
def train_test_split(df):
    
    # compute split point
    end_idx = df.shape[0]* 70 // 100
    
    train_data = df.iloc[:end_idx, : ]
    test_data = df.iloc[end_idx:, :]
    
    return train_data, test_data

In [9]:
# Split the data into train and test
X_train, X_test = train_test_split(df)

In [10]:
X_test.shape
X_train.shape

(1009, 8)

In [11]:
from sklearn.preprocessing import MinMaxScaler
def scale_data(train, test):
    scaler = MinMaxScaler().fit(train)
    return scaler.transform(train), scaler.transform(test), scaler

In [12]:
# Scale the data
X_train, X_test, scaler = scale_data(X_train, X_test)

In [13]:
def convert_to_supervised(df):

    input_features = []
    ouput_feature = []
    
    len_df = df.shape[0]
    
    for i in range(len_df):
        
        end_idx = i + 1 
        
        if end_idx > len_df-1:
            break
            
        input_x , output_y = df[i:end_idx, 1:], df[end_idx: end_idx+1, 0]
        
        input_features.append(input_x)
        ouput_feature.append(output_y)
    
    return np.array(input_features), np.mean(np.array(ouput_feature), axis=1)

In [14]:
# Split the training data into input features and out feature
X_train, Y_train = convert_to_supervised(X_train)
print('Shape of (training data) input features : %s and ouput feature %s' % (X_train.shape, Y_train.shape))

Shape of (training data) input features : (1008, 1, 7) and ouput feature (1008,)


In [15]:
# Split the testing data into input features and out feature
X_test, Y_test = convert_to_supervised(X_test)
print('Shape of (testing data) input features : %s and ouput feature %s' % (X_test.shape, Y_test.shape))

Shape of (testing data) input features : (432, 1, 7) and ouput feature (432,)


In [16]:
import tensorflow as tf
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

## CNN-LSTM

In [18]:
cnn_lstm_model = CNN_LSTM.CNN_LSTM(1,7,1).getModel()
cnn_lstm_model.compile(optimizer="adam",loss="mse")

In [19]:
cnn_lstm_model.fit(X_train, Y_train, epochs= 100, verbose= 1 ,callbacks=callback)

Epoch 1/100
32/32 [==============================] - 2s 3ms/step - loss: 0.0303
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0160
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0141
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0144
Epoch 5/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0139
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0132
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0129
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0128
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0129
Epoch 10/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0123
Epoch 11/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0126
Epoch 12/100
32/32 [==============================] - 0s 3ms/step - loss: 0.0124
Epoch 13/100
32/32 [=================

In [20]:
Evaluation_Metrix.print_metrics(Y_train, 
                                cnn_lstm_model.predict(X_train),
                                Y_test,
                                cnn_lstm_model.predict(X_test))

14/14 [==============================] - 0s 2ms/step
Train RMSE value   : 0.103 
Train MSE value    : 0.011 
Train R2 value     : 0.478 
Train MAE value    : 0.077 
---------------------------
Test RMSE value    : 0.080 
Test MSE value     : 0.006 
Test R2 value      : 0.416 
Test MAE value     : 0.062 
